This script calculates the Climate Shelter Index for each public green area.  
A climate shelter should have these characteristics:
- Surface area > 0.5 ha 
- NDVI > 0.4 
- Good accessibility
- Presence of drinking fountains 
- Presence of  benches 
- Presence of picnic tables

### 0. Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import geopandas as gpd

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### 1. Climate Shelter Index - CSI

In [ ]:
# Load your shapefile
green_area = gpd.read_file('data\Climate_Shelter_Index\isocrona\access_index')

In [41]:
green_area.head()

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,benches,id_bench,NDVI,n_cycleway,n_footway,busstop5,busstop10,disab_p_5,access_ind,geometry
0,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,9509.007,12461.168,2009-01-29,...,no,N/A,0.542155,0,0,2,3,-1,0.012821,"POLYGON ((11.31509 44.42248, 11.31511 44.42256..."
1,Santo Stefano,GIARDINO MONUMENTO CADUTI DI SABBIUNO,Via di Sabbiuno,PARCO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,5514.205,6730.89,2009-01-29,...,no,N/A,0.459359,0,0,0,0,-1,0.000000,"POLYGON ((11.31288 44.42104, 11.31289 44.42109..."
2,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,16811.194,24817.745,2007-03-13,...,no,N/A,0.624308,0,0,2,3,-1,0.012821,"POLYGON ((11.31463 44.42146, 11.31496 44.42149..."
3,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,87.272,6250.574,2007-09-20,...,no,N/A,0.877684,0,8,3,8,-1,0.071077,"POLYGON ((11.32362 44.45097, 11.32372 44.45110..."
4,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,-4.817,21360.397,2007-09-20,...,no,N/A,0.879811,0,8,3,8,-1,0.071077,"POLYGON ((11.32160 44.45340, 11.32164 44.45341..."


In [42]:
green_area.columns

Index(['quart', 'nome', 'ubicazione', 'classe_uni', 'classe_gia', 'classe_pen',
       'classe_suo', 'area_prato', 'area_ug', 'data_agg', 'siepi', 'bosco',
       'arboreo', 'arbustivo', 'pavim', 'idro', 'quartiere', 'area_stati',
       'zona_pross', 'area_Ha', 'd_fountain', 'id_df', 'picnic_tab', 'id_pt',
       'benches', 'id_bench', 'NDVI', 'n_cycleway', 'n_footway', 'busstop5',
       'busstop10', 'disab_p_5', 'access_ind', 'geometry'],
      dtype='object')

#### 1.1 Determine weights with PCA

In [43]:
# Assuming 'id_df', 'id_bench', and 'id_pt' contain lists of ids, we will count them.

green_area['n_df'] = green_area['id_df'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)
green_area['n_bench'] = green_area['id_bench'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)
green_area['n_pt'] = green_area['id_pt'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)
green_area['norm_area'] = green_area['area_Ha'].apply(lambda x: x if pd.notna(x) else 0) # substitue NA with 0
green_area['norm_area'] = green_area['norm_area'].apply(lambda x: x if x > 0 else 0)
green_area['norm_NDVI'] = green_area['NDVI'].apply(lambda x: x if pd.notna(x) else 0)   # substitue NA with 0

# Print to verify the new columns
print(green_area[[ 'n_df', 'n_bench', 'n_pt', 'norm_area', 'norm_NDVI']].head() )      

   n_df  n_bench  n_pt  norm_area  norm_NDVI
0     1        1     1   0.950901   0.542155
1     1        1     1   0.551420   0.459359
2     1        1     1   1.681119   0.624308
3     1        1     1   0.008727   0.877684
4     1        1     1   0.000000   0.879811


In [44]:
# Step 1: Ensure that only necessary columns are used
pca_green_area = green_area[['n_df', 'n_bench', 'n_pt', 'norm_area', 'norm_NDVI', 'access_ind']]

# Step 2: Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(pca_green_area[['n_df', 'n_bench', 'n_pt', 'norm_area', 'norm_NDVI', 'access_ind']])

# Step 3: Perform PCA
pca = PCA()
pca.fit(scaled_data)

# Step 4: Analyze PCA output
print("Variance explained by each component:", pca.explained_variance_ratio_)
print("Cumulative variance explained:", np.cumsum(pca.explained_variance_ratio_))

# Step 5: Extract and visualize PCA loadings
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)


Variance explained by each component: [0.4363023  0.18933128 0.1552898  0.13092822 0.07959903 0.00854938]
Cumulative variance explained: [0.4363023  0.62563358 0.78092337 0.9118516  0.99145062 1.        ]


In [45]:
# Step 6: create df with loadings
loadings_df = pd.DataFrame(loadings, columns=[f'PC{i+1}' for i in range(len(pca_green_area.columns))], index=pca_green_area.columns)
print(loadings_df)

                 PC1       PC2       PC3       PC4       PC5       PC6
n_df        0.942863 -0.109053 -0.096528  0.035890  0.252238 -0.161133
n_bench     0.940576 -0.096979 -0.099497  0.025585  0.266575  0.159283
n_pt        0.734405 -0.239268 -0.156975 -0.306676 -0.534583  0.002355
norm_area   0.421594  0.629590 -0.024710  0.613665 -0.223059  0.001336
norm_NDVI   0.147109  0.812494  0.001679 -0.559531  0.078758 -0.002915
access_ind  0.328974 -0.046200  0.942481 -0.027616 -0.041050  0.000745


In [46]:
#Step 7: Calculate normalized weights
weights = np.abs(loadings_df['PC1']) / np.sum(np.abs(loadings_df['PC1']))
print(weights)

n_df          0.268200
n_bench       0.267550
n_pt          0.208904
norm_area     0.119923
norm_NDVI     0.041846
access_ind    0.093578
Name: PC1, dtype: float64


In [47]:
# Define weights for each characteristic (surface area, NDVI, accessibility, drinking fountains, benches, picnic tables)
weights_dict = weights.round(2).to_dict()
print(weights_dict)

{'n_df': 0.27, 'n_bench': 0.27, 'n_pt': 0.21, 'norm_area': 0.12, 'norm_NDVI': 0.04, 'access_ind': 0.09}


In [48]:
# Define the columns you want to normalize
columns_to_normalize = ['n_df', 'n_bench', 'n_pt']

# Perform min-max normalization for each column
for col_name in columns_to_normalize:
    min_value = green_area[col_name].min()
    max_value = green_area[col_name].max()
    green_area[col_name] = (green_area[col_name] - min_value) / (max_value - min_value)


In [49]:
# Normalize surface area (0.5 ha threshold)
green_area['norm_area'] = green_area['norm_area'].apply(lambda x: 1 if x >= 0.5 else 0)


In [50]:
# Normalize NDVI (0.4 threshold)
green_area['norm_NDVI'] = green_area['norm_NDVI'].apply(lambda x: x if x >= 0.4 else 0)


In [51]:
# Calculate the CSI based on the weighted and normalized variables
green_area['CSI'] = (
    weights_dict['norm_area'] * green_area['norm_area'] +
    weights_dict['norm_NDVI'] * green_area['norm_NDVI'] +
    weights_dict['access_ind'] * green_area['access_ind'] +
    weights_dict['n_df'] * green_area['n_df'] +
    weights_dict['n_bench'] * green_area['n_bench'] +
    weights_dict['n_pt'] * green_area['n_pt']
)

In [52]:
green_area = green_area.drop(columns=['n_df', 'n_bench', 'n_pt', 'norm_area', 'norm_NDVI'])

In [53]:
green_area[green_area['NDVI']<0.41].head()

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,id_bench,NDVI,n_cycleway,n_footway,busstop5,busstop10,disab_p_5,access_ind,geometry,CSI
220,Borgo Panigale - Reno,GIARDINO TERESA NOCE,Via del Beccaccino,GIARDINO,PIANO,NaN,ARGILLOSO COMPATTO,0.114,239.302,2015-06-10,...,N/A,0.398521,0,5,9,34,22,0.125866,"POLYGON ((11.29106 44.49354, 11.29112 44.49352...",0.011328
311,Porto - Saragozza,SCUOLA MEDIA GANDINO,Via Graziano,VERDE SCOLASTICO,PIANO,0 -20%,ARGILLOSO COMPATTO,55.935,55.935,2006-06-06,...,N/A,0.388157,0,0,0,0,-1,0.000000,"POLYGON ((11.33103 44.50035, 11.33104 44.50038...",0.000000
317,Porto - Saragozza,SCUOLA ELEM. TERESINA GUIDI,Via Calori,VERDE SCOLASTICO,PIANO,0 -20%,ARGILLOSO COMPATTO,30.760,30.76,2006-06-06,...,N/A,0.378452,0,0,7,38,40,0.102983,"POLYGON ((11.33043 44.50045, 11.33043 44.50046...",0.009268
330,Santo Stefano,GIARDINO LEGNANI PIZZARDI,Via del Cane 3/b,GIARDINO,PIANO,0 -20%,MEDIO IMPASTO,3.783,7.242,2009-10-20,...,N/A,0.351903,0,0,0,0,-1,0.000000,"POLYGON ((11.34252 44.49087, 11.34252 44.49086...",0.000000
332,Santo Stefano,GIARDINO LEGNANI PIZZARDI,Via del Cane 3/b,GIARDINO,PIANO,0 -20%,MEDIO IMPASTO,3.480,6.502,2009-10-20,...,N/A,0.365648,0,0,0,0,-1,0.000000,"POLYGON ((11.34255 44.49084, 11.34256 44.49085...",0.000000


In [ ]:
green_area.to_file('data\Climate_Shelter_Index\isocrona\CSI')

#### 1.2 CSI for selected areas


In [ ]:
csi = gpd.read_file('data\Climate_Shelter_Index\isocrona\CSI')
covered_areas = gpd.read_file("data\Meteoblue\covered_areas.shp")

In [4]:
# select only green area within covered areas
covered_areas_csi =csi[csi.geometry.within(covered_areas.unary_union)] 

In [5]:
covered_areas_csi.columns

Index(['quart', 'nome', 'ubicazione', 'classe_uni', 'classe_gia', 'classe_pen',
       'classe_suo', 'area_prato', 'area_ug', 'data_agg', 'siepi', 'bosco',
       'arboreo', 'arbustivo', 'pavim', 'idro', 'quartiere', 'area_stati',
       'zona_pross', 'area_Ha', 'd_fountain', 'id_df', 'picnic_tab', 'id_pt',
       'benches', 'id_bench', 'NDVI', 'n_cycleway', 'n_footway', 'busstop5',
       'busstop10', 'disab_p_5', 'access_ind', 'CSI', 'geometry'],
      dtype='object')

In [6]:
covered_areas_csi_sorted = covered_areas_csi.sort_values(by='CSI', ascending=False).head(5)

In [5]:
for index, row in covered_areas_csi_sorted.iterrows():
    print(f"{row['nome']}, with CSI of {round(row['CSI'],3)}")

PARCO DELLA MONTAGNOLA, with CSI of 0.933
GIARDINO CENTRO CIVICO SAN DONATO (EX BENTIVOGLI E MARCINELLE), with CSI of 0.861
PARCO NICHOLAS GREEN ( EX  VILLA CONTRI), with CSI of 0.717
PARCO DI VILLA ANGELETTI, with CSI of 0.712
P.CO LUNETTA GAMBERINI, with CSI of 0.692


In [60]:
covered_areas_csi_sorted.explore()

In [ ]:
covered_areas_csi.to_file('data\Climate_Shelter_Index\CSI_covered_areas')

In [ ]:
covered_areas_csi_sorted.to_file('data\Climate_Shelter_Index\CSI_covered_areas_top5')

##### 1.2.1 Green Surface > 0.5

In [63]:
covered_areas_csi[covered_areas_csi['area_Ha']>0.5].shape

(181, 35)

In [64]:
# find the top 10 parks with the largest surface area
covered_areas_csi_sorted_2 = covered_areas_csi.sort_values(by='area_Ha', ascending=False).head(10)

In [65]:
for index, row in covered_areas_csi_sorted_2.iterrows():
    print(f"{row['nome']}, with a green area of {round(row['area_Ha'], 3)} hectares")

GIARDINI MARGHERITA, with a green area of 14.934 hectares
LUNGO RENO CHIARINI BERTOCCHI, with a green area of 8.194 hectares
PARCO VINCENZO TANARA, with a green area of 6.644 hectares
P.CO LUNETTA GAMBERINI, with a green area of 6.435 hectares
PARCO BADEN POWELL, with a green area of 6.123 hectares
PARCO DI VILLA ANGELETTI, with a green area of 5.954 hectares
PARCO DELLE QUERCE, with a green area of 5.446 hectares
PARCO NICHOLAS GREEN ( EX  VILLA CONTRI), with a green area of 5.046 hectares
LUNGORENO PONTE BACCHELLI - PONENTE, with a green area of 4.755 hectares
LUNGORENO TRATTO TRIUMVIRATO, with a green area of 4.412 hectares


##### 1.2.2 NDVI > 0.4

In [66]:
covered_areas_csi[covered_areas_csi['NDVI']>0.4].shape

(702, 35)

##### 1.2.3 NDVI + Green Surface

In [67]:
covered_areas_csi[(covered_areas_csi['NDVI']>0.4) & (covered_areas_csi['area_Ha']>0.5)].shape

(181, 35)